# HTTP Requests
There are many types of HTTP requests. The four most commonly used types of HTTP requests are GET, POST, PUT, and DELETE.

## Get Requests
The first type of requests we’re going to tackle are GET requests using fetch(). The `fetch()` function:

- Creates a request object that contains relevant information that an API needs.
- Sends that request object to the API endpoint provided.
- Returns a promise that ultimately resolves to a response object, which contains the status of the promise with information the API sent back.

The basic structure of a get request is shown via the examples below with the [Datamuse API](https://www.datamuse.com/api/), each with a different query use-case. Additionally, another example for [the movie database](https://www.themoviedb.org/) api is shown.

The first api use-case is used to get back an array of words that sound like the word we typed into the input field. 

In [1]:
var fetch = require('node-fetch');

// Information to reach API
var url = "https://api.datamuse.com/words?" // api url
var useCase = "sl=" // words that sound like <query> 

// Asynchronous function
var getSuggestions = (url, useCase, query) => {
  var endpoint = url + useCase + query;

  fetch(endpoint)
    .then((response) => { // sends request

      if (response.ok) {
          return response.json() // returns promise
      }

      throw new Error('Request failed!') // handles errors

    }, (networkError) => {

      console.log(networkError.message) // log error if thrown

    })
  
    .then(jsonResponse => { // handles success
      console.log(jsonResponse) // log json object
    })
}

getSuggestions(url, useCase, 'hipopotamus')

[ { word: 'hippopotamus', score: 100, numSyllables: 5 },
  { word: 'hippopotami', score: 85, numSyllables: 5 },
  { word: 'hippopotamuses', score: 80, numSyllables: 6 },
  { word: 'hippopotame', score: 80, numSyllables: 4 },
  { word: 'hypodermis', score: 60, numSyllables: 4 },
  { word: 'habitance', score: 55, numSyllables: 3 },
  { word: 'saprostomous', score: 55, numSyllables: 4 },
  { word: 'synonymous', score: 40, numSyllables: 4 },
  { word: 'sopapilla', score: 38, numSyllables: 4 },
  { word: 'sea bottom', score: 38, numSyllables: 3 } ]


The second use-case finds words that are likely within the context of the adjective used for the query.

In [2]:
var fetch = require('node-fetch');

// Information to reach API
var url = "https://api.datamuse.com/words?" // api url
var useCase = 'rel_jja=' // nouns that are often described by the <query adjective>

// Asynchronous function
var getSuggestions = async (url, useCase, query) => {
  var endpoint = url + useCase + query;
  try {
    var response = await fetch(
      endpoint,
    )
    if (response.ok) {
      var jsonResponse = await response.json()
      var cap = Math.min(jsonResponse.length, 10) // caps responses at 10
      var results = jsonResponse.slice(0, cap)
      console.log(results)
    }

  } catch(error) {
    console.log(error)
  }
}
  
getSuggestions(url, useCase, 'grand')

Promise { <pending> }

[ { word: 'jury', score: 1001 },
  { word: 'scale', score: 1000 },
  { word: 'total', score: 999 },
  { word: 'piano', score: 998 },
  { word: 'style', score: 997 },
  { word: 'design', score: 996 },
  { word: 'master', score: 995 },
  { word: 'strategy', score: 994 },
  { word: 'duke', score: 993 },
  { word: 'manner', score: 992 } ]


The below example demonstrates a GET request to [the movie database](https://www.themoviedb.org/) api.

In [3]:
var tmdbBaseUrl = 'https://api.themoviedb.org/3'
var fetch = require('node-fetch');

try {
  var {tmdb_rat} = require('./keys.js') // import random access token
} catch (err) {
  console.error('Error reading file:', err.message); // log error if any
}

var getGenres = async () => {
    var genreRequestEndpoint = '/genre/movie/list?language=en'
    var urlToFetch = tmdbBaseUrl + genreRequestEndpoint
    var options = {
      method: 'GET',
      headers: {
        accept: 'application/json',
        Authorization: `Bearer ${tmdb_rat}`
      }
    }  
    try {
      var response = await fetch(urlToFetch, options)
  
      if (response.ok) {
        var jsonResponse = await response.json()
        console.log(jsonResponse)
      }
  
    } catch(error) {
      console.log(error)
    }
  }

getGenres()

Promise { <pending> }

{ genres: 
   [ { id: 28, name: 'Action' },
     { id: 12, name: 'Adventure' },
     { id: 16, name: 'Animation' },
     { id: 35, name: 'Comedy' },
     { id: 80, name: 'Crime' },
     { id: 99, name: 'Documentary' },
     { id: 18, name: 'Drama' },
     { id: 10751, name: 'Family' },
     { id: 14, name: 'Fantasy' },
     { id: 36, name: 'History' },
     { id: 27, name: 'Horror' },
     { id: 10402, name: 'Music' },
     { id: 9648, name: 'Mystery' },
     { id: 10749, name: 'Romance' },
     { id: 878, name: 'Science Fiction' },
     { id: 10770, name: 'TV Movie' },
     { id: 53, name: 'Thriller' },
     { id: 10752, name: 'War' },
     { id: 37, name: 'Western' } ] }


## POST Requests
The `fetch()` call takes two arguments: an endpoint and an object that contains information needed for the request. This object includes a method. In this case, we'll pass a value of 'POST'. For the request body, you'll need to pass a JSON string, so the JSON.stringify function will come in handy. This includes the data sent to the API.

A successful POST request will return a response body, which will vary depending on how the API is set up. The rest of the request is identical to the GET request. A .then() method is chained to the fetch() function to check and return the response as well as throw an exception when a network error is encountered. A second .then() method is added on so that we can use the response however we may choose.

The example below demonstrates a POST request to the rebrandly api. It offers developers a way to create a short but superpowered url that enables link analytics. To follow along with the example below, you'll need to make an account on [rebrandly](https://www.rebrandly.com/) and create an api key.

In [ ]:
var fetch = require('node-fetch');

// Asynchronous function
var shortenUrl = (urlToShorten, apiKey) => {
    var data = JSON.stringify({destination: urlToShorten}) 
      
    fetch('https://api.rebrandly.com/v1/links', {
        method: 'POST',
        headers: {
            'Content-type': 'application/json',
            'apikey': apiKey
        },
        body: data
    })
   .then(response => {
        if (response.ok) {
          console.log(response.ok)
          return response.json() // returns promise for json data
        }
    
        throw new Error('Request failed!') // log if no response
    
      }, networkError => {
        console.log(networkError.message) // log network error if any
      })
    .then(jsonResponse => {
      console.log(jsonResponse) // log shortend url
    })
  }

try {
  var {rebrandly} = require('./keys.js') // import api key
} catch (err) {
  console.error('Error reading file:', err.message); // log error if any
}

var urlToShorten = 'https://github.com/jgome284';

shortenUrl(urlToShorten, rebrandly) // returns shortend url